In [138]:
from timeit import default_timer as timer
from functools import lru_cache


f = open("data.txt", "r")
data = f.read()
rules, messages = data.split('\n\n')
rules = rules.split('\n')
messages = messages.split('\n')
# data = [x.strip('\n') for x in data]



In [139]:
# Part 1 Monster Messages
# why use re when I can just do something simple like NFAs myself?????

# parse input
def parseRules(rules):
    table = {}

    for rule in rules:
        state, transFunction = rule.split(': ')
        # the transition function can be a letter, 1 or 2 states, or between a set of two states
        if '\"' in transFunction: # a letter
            table[state] = transFunction.strip('\"')
        else:
            # table[state] = transFunction.split(' | ')

            table[state] = [x.split(' ') for x in transFunction.split(' | ')]
        


        # print(state)
        # print(transFunction)
    return table
table = parseRules(rules)
table['0']

[['8', '11']]

In [140]:
# string: aaabbbb
# rule: 1 2
# how do we know where rule 1 ends??
# we can expand the range we check until it works or doesnt


In [141]:
# check if a  single rule maps to a char if not then it goes back to checkWholeRule with the next rule
# message is a substring of orginal message
# partialRule is just a single rule
@lru_cache(maxsize=None) 
def checkPartialRule(message, partialRule):
    # print(checkPartialRule.__name__, f'message={message}, partialRule={partialRule}')
    # get the rule for partialRule
    rule = table[partialRule]
    # print(f'\trule={rule}')
    # print(rule)
    if len(rule) == 1 and rule[0][0].isalpha():
        # print(f'returns {message == rule}')
        return message == rule
    else:
        # print(f'returns checkWholeRule')
        return checkWholeRule(message, rule)

In [151]:
def checkWholeRule(message, rules):
    """
    Checks if message string conforms to a set of rules. 

        Parameters:
            message (str): A string containing  
            rules ([[str...]...]): List containing lists of rules
        
        Returns:
            (bool): True if message matches rules

    """

    # checkWholeRule.cache 
    # print(checkWholeRule.count)

    # print(message, rules)


    # print(checkWholeRule.__name__, f'message={message}, rules={rules}')

    for rule in rules:
        lookupStr = f' '.join(rule) + ' ' + message
        if lookupStr in checkWholeRule.cache:
            return checkWholeRule.cache[lookupStr]

        # print(lookupStr)
        # keep track of the last index of each partial rule
        if len(rule) > len(message):
            # print('\tFALSE: len(rule) > len(message)')
            checkWholeRule.cache[lookupStr] = False
            return False
        # ruleWindows = list(reversed(range(len(rule))))
        # ruleWindows = list(map(lambda x: len(message) - x, ruleWindows))
        # calcWindowSizes(ruleWindows)

        ruleWindows = list(range(len(rule)))
        ruleWindows[-1] = len(message)
        ruleWindows = list(map(lambda x: x+1, ruleWindows))
        for currRuleWindow in calcWindowSizes(ruleWindows):
            # print(checkWholeRule.__name__, f'message={message}, rules={rules}')
            # print(f'\tcurrRuleWindow={currRuleWindow}')
            # check if the current rule windows are valid
            prevRW = 0
            validRules = True
            print(currRuleWindow, rule)
            for rw, r in zip(currRuleWindow, rule):
                # print(f'\trw={rw}, r={r}')
                if not checkPartialRule(message[prevRW:rw], r):
                    validRules = False
                    break
                prevRW = rw
            if validRules:
                checkWholeRule.cache[lookupStr] = True
                return True
            else:
                checkWholeRule.cache[lookupStr] = False
    return False
checkWholeRule.cache = {}


In [152]:
# damn no good tools to do this
# guess we gotta write out own
def calcWindowSizes(ruleWindows):
    '''
    yields a generator that iterates through the possible window end indicies for each rule
    calcWindowSizes([1,2,4]) => [1, 2, 4], [1, 2, 4], [1, 3, 4] ,[2, 3, 4]
    '''
    rw = ruleWindows[:]
    currRW = ruleWindows[:]
    # we dont know how many elements are in ruleWindows by default this
    # the first time we want to yield the currrent ruleWindows
    yield rw
    if len(rw) > 1:
        for i in range(1,rw[-1]-len(rw)+2):
            currRW[0] = i
            yield from calcWindowSizesHelper(currRW, ruleWindows[1:], 1)


def calcWindowSizesHelper(currRW, ruleWindows, index):
    # print(ruleWindows)
    # print("two", ruleWindows[0])
    if len(ruleWindows) > 1:
        curr = currRW[index-1]+1
        while curr < currRW[index+1]:
            currRW[index] = curr
            # currRW[index+1] = currRW[index+1] +1
            yield from calcWindowSizesHelper(currRW, ruleWindows[1:], index+1)
            curr += 1
    else:
        yield currRW


In [153]:
# main function that iterates over all the messages
def checkMessages(messages):
    count = 0
    for message in messages:
        if checkWholeRule(message, table['0']):
            count += 1
    return count

In [154]:
# takes a hot sec to run

#part 1
# print(f'part1={checkMessages(messages)}')

# # part 2
table['8'] = [['42','8'],['42']]
table['11'] = [['42','11','31'],['42','31']]



start = timer()
print(f'part2={checkMessages(messages)}')
end = timer()
print(checkMessages.__name__, end - start)


part2=68
checkMessages 2.423536522999939


In [147]:
checkPartialRule.cache_info()

CacheInfo(hits=10930502, misses=430278, maxsize=None, currsize=430278)